# 네이버 영화 리뷰 데이터 분류

### 전처리 과정

In [65]:
import pandas as pd
import urllib.request

# 데이터셋 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x2d0221e0220>)

In [66]:
import pandas as pd

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [67]:
# 영화 리뷰 개수 확인
print('train:', len(train_data))
print('test:', len(test_data))

train: 150000
test: 50000


In [68]:
# 상위 5개 출력하여 데이터 확인
train_data.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [69]:
# 라벨의 분포 확인
train_data['label'].value_counts()

label
0    75173
1    74827
Name: count, dtype: int64

In [70]:
# document 열에서 중복인 내용이 있다면 중복 제거
len(train_data['document'].unique())
train_data.drop_duplicates(subset=['document'], inplace=True)

print('제거 후 남은 샘플 수: ', len(train_data))

제거 후 남은 샘플 수:  146183


In [71]:
# 데이터 내에 결측치가 있습니다. 찾아서 제거주세요.
# 결측치 확인
print(train_data.isnull().sum())
print(train_data.loc[train_data.document.isnull()])

# 결측치 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거

print(len(train_data)) # 146182

id          0
document    1
label       0
dtype: int64
            id document  label
25857  2172111      NaN      1
146182


한글만 남기고 나머지 데이터 제거

In [72]:
import numpy as np

train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # 시작 부분의 공백 제거
train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how = 'any') # Null 값 제거
train_data.head(10)

C:\Users\qq\AppData\Local\Temp\ipykernel_13364\4222784049.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [73]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 시작 부분의 공백 제거
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
test_data.head(10)

C:\Users\qq\AppData\Local\Temp\ipykernel_13364\1803854010.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


In [74]:
print('전처리 후 테스트용 샘플의 개수 :',len(train_data))
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 146182
전처리 후 테스트용 샘플의 개수 : 49157


In [75]:
# 불용어 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', 'ㅋ']

In [76]:
from konlpy.tag import Okt
okt = Okt()

In [77]:
########## 시간이 많이 소요됩니다 ############
from tqdm import tqdm
X_train = []

# `tqdm`을 사용하여 문장 처리의 진행 상황을 표시합니다.
for sentence in tqdm(train_data['document']):
    # 문장을 형태소 단위로 토큰화, `stem=True`는 형태소의 원형 반환
    tokenized_sentence = okt.morphs(sentence, stem=True)
    
    # 불용어 제거, 불용어 리스트에 포함되지 않은 단어만 남김
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    
    # 불용어가 제거된 단어 리스트
    X_train.append(stopwords_removed_sentence)

# 최종적으로, `X_train` 리스트에는 전처리된 문장들이 토큰화되고 불용어가 제거된 형태로 저장됩니다.

100%|██████████| 146182/146182 [09:03<00:00, 269.07it/s]


In [78]:
########## 시간이 많이 소요됩니다 ############
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 49157/49157 [03:16<00:00, 249.55it/s]


In [79]:
print('전처리 후 테스트용 샘플의 개수 :',len(X_test))

전처리 후 테스트용 샘플의 개수 : 49157


In [80]:
X_train_new = X_train
X_test_new = X_test

In [81]:
#X_train = X_train_new
#X_test = X_test_new

In [82]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 단어 집합 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [83]:
print(tokenizer.word_index)

{'.': 1, '영화': 2, '보다': 3, '..': 4, '을': 5, '...': 6, ',': 7, '없다': 8, '이다': 9, '있다': 10, '좋다': 11, '너무': 12, '?': 13, '다': 14, '정말': 15, '되다': 16, '재밌다': 17, '적': 18, '만': 19, '같다': 20, '진짜': 21, '!': 22, '로': 23, '점': 24, '아니다': 25, '않다': 26, '에서': 27, '만들다': 28, '연기': 29, '나오다': 30, '것': 31, '평점': 32, '내': 33, '~': 34, '최고': 35, '그': 36, '나': 37, '안': 38, '인': 39, '스토리': 40, '생각': 41, '못': 42, '....': 43, '왜': 44, '드라마': 45, '게': 46, '감동': 47, '사람': 48, '1': 49, '보고': 50, '이렇다': 51, '말': 52, '고': 53, '아깝다': 54, '더': 55, '아': 56, '때': 57, '배우': 58, 'ㅋㅋ': 59, '거': 60, '감독': 61, '그냥': 62, '재미있다': 63, '요': 64, '내용': 65, '재미': 66, '중': 67, '뭐': 68, '까지': 69, '!!': 70, '주다': 71, '자다': 72, '하고': 73, '지루하다': 74, '재미없다': 75, '시간': 76, '수': 77, '쓰레기': 78, '가다': 79, '네': 80, '모르다': 81, '들다': 82, '그렇다': 83, '싶다': 84, '지': 85, '작품': 86, '10': 87, '사랑': 88, '알다': 89, '하나': 90, '다시': 91, '마지막': 92, '볼': 93, '이건': 94, '오다': 95, '저': 96, '정도': 97, '완전': 98, '2': 99, 'ㅋㅋㅋ': 100, '많다': 101, '처음': 102,

In [84]:
print(tokenizer.word_counts.items())

odict_items([('아', 4359), ('더빙', 573), ('..', 28928), ('진짜', 8317), ('짜증나다', 1008), ('목소리', 374), ('흠', 259), ('...', 22611), ('포스터', 573), ('보고', 4637), ('초딩', 422), ('영화', 50461), ('줄', 1240), ('....', 5177), ('오버', 142), ('연기', 6432), ('조차', 242), ('가볍다', 361), ('않다', 7714), ('너', 686), ('무재', 69), ('밓었', 1), ('다그', 24), ('래서', 17), ('보다', 41094), ('추천', 1187), ('다', 10188), ('교도소', 16), ('이야기', 2171), ('구먼', 11), ('솔직하다', 1204), ('재미', 3888), ('없다', 15568), ('평점', 6275), ('조정', 40), ('사이', 219), ('몬페', 2), ('그', 5669), ('익살스럽다', 9), ('돋보이다', 288), ('!', 8288), ('스파이더맨', 63), ('에서', 6990), ('늙다', 197), ('보이다', 1071), ('커스틴', 4), ('던스트', 2), ('너무나도', 206), ('이쁘다', 1033), ('막', 330), ('걸음', 20), ('마', 238), ('떼다', 151), ('3', 2233), ('세', 632), ('부터', 1697), ('초등학교', 116), ('1', 4733), ('학년', 122), ('생인', 5), ('8', 535), ('살다', 1169), ('.', 67405), ('ㅋㅋㅋ', 2459), ('별', 2010), ('반개', 211), ('아깝다', 4402), ('움', 1055), ('원작', 1252), ('긴장감', 830), ('을', 23183), ('제대로', 799), ('살리다', 533),

사용되는 단어 수가 지나치게 많아 보임 -> 줄일 필요 있음

In [85]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 49585
등장 빈도가 2번 이하인 희귀 단어의 수: 28787
단어 집합에서 희귀 단어의 비율: 58.05586366844812
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.9119459785408355


In [86]:
# 단어수 결정(전체 - 빈도수 2이하의 수 + padding(1))
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 20799


In [87]:
tokenizer = Tokenizer(vocab_size) # 빈도수 2 이하인 단어는 제거
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [88]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [89]:
# 학습용 데이터와 라벨 개수 확인
print(len(X_train))
print(len(y_train))

# 단어가 정수로 변환된 것 확인
print(X_train[:5])

146182
146182
[[56, 486, 4, 21, 279, 703], [962, 6, 487, 50, 648, 2, 232, 43, 1528, 29, 1012, 721, 26], [404, 2571, 5302, 6618, 3, 239, 14], [6851, 118, 8575, 4, 235, 66, 8, 4, 32, 3794], [1097, 36, 9661, 29, 879, 2, 22, 2736, 27, 1174, 258, 15125, 1140, 271, 258]]


빈도수가 낮은 데이터를 제거하며, 빈 행이 생길 가능성이 있음

In [90]:
# 빈도수가 낮은 단어로만 구성된 행은 빈 샘플이 되었을 수 있음(길이가 1미만인 행 찾기)
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

In [91]:
# 빈 샘플 확인
print(len(drop_train))
print(len(drop_test))

307
134


In [95]:
# X_train 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
X_test = np.delete(X_test, drop_test, axis=0)
y_test = np.delete(y_test, drop_test, axis=0)

In [96]:
# 빈 샘플 제거 후 길이 확인
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

145875 49023
145875 49023


In [97]:
import matplotlib as plt

print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
#plt.hist([len(s) for s in X_train], bins=50)
#plt.xlabel('length of samples')
#plt.ylabel('number of samples')
#plt.show()

리뷰의 최대 길이 : 36
리뷰의 평균 길이 : 36.0


In [98]:
# 95%의 데이터를 포함하는 샘플의 길이 결정
lengths = []

for x in X_train:
    length = len(x)
    lengths.append(length)

max_len = int(np.percentile(lengths, 95))
print(max_len)

36


In [94]:
# padding, 전체 데이터의 길이를 max_len 으로 맞춘다
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# 2. LSTM으로 네이버 영화 리뷰 감성 분류하기

In [107]:
# 콜백 생성
from tensorflow.keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model_naver_movie.keras', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, Dense

# 모델 생성 및 컴파일
model = Sequential()
model.add(Embedding())
model.add(Flatten())
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [109]:
# 모델 학습
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[mc])

Epoch 1/5
897/912 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - acc: 0.4972 - loss: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to best_model_naver_movie.keras
912/912 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.4972 - loss: 0.0000e+00 - val_acc: 0.4962 - val_loss: 0.0000e+00
Epoch 2/5
849/912 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4987 - loss: 0.0000e+00
Epoch 2: val_loss did not improve from 0.00000
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.4986 - loss: 0.0000e+00 - val_acc: 0.4962 - val_loss: 0.0000e+00
Epoch 3/5
856/912 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - acc: 0.4983 - loss: 0.0000e+00
Epoch 3: val_loss did not improve from 0.00000
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.4983 - loss: 0.0000e+00 - val_acc: 0.4962 - val_loss: 0.0000e+00
Epoch 4/5
850/912 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - acc: 0.4998 - loss: 0.0000e+00
Epoch 4: val_loss did not improve from 0.00000
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.4997 - loss: 0.0000e+00 - val

In [117]:
# 정확도 평가
result = model.evaluate(X_test, y_test)

print("\n테스트 손실: %.4f" % (result[0]))
print("테스트 정확도: %.4f" % (result[1]))

1532/1532 ━━━━━━━━━━━━━━━━━━━━ 1s 716us/step - acc: 0.5041 - loss: 0.0000e+00

테스트 손실: 0.0000
테스트 정확도: 0.5027


# 저장된 가중치 불러와서 리뷰 예측해보기

In [118]:
loaded_model = load_model('best_model_naver_movie.keras')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

NameError: name 'load_model' is not defined

In [119]:
import re
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [112]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

NameError: name 'loaded_model' is not defined

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')